## Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
from pyspark.sql import Window
import plotly.express as px

## Data Ingestion

In [2]:
filepaths = "../data/CTU-IoT-Malware-Capture-1-1conn.log.labeled.csv"
spark = SparkSession.builder.appName("anomaly_detection").getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
spark.sparkContext.version

24/01/30 16:32:38 WARN Utils: Your hostname, Ayodejis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.160.70 instead (on interface en0)
24/01/30 16:32:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/30 16:32:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.5.0'

In [4]:
df.show()

+-----------------+------------------+---------------+---------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+
|               ts|               uid|      id.orig_h|id.orig_p|      id.resp_h|id.resp_p|proto|service|duration|orig_bytes|resp_bytes|conn_state|local_orig|local_resp|missed_bytes|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|tunnel_parents|    label|      detailed-label|
+-----------------+------------------+---------------+---------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+
|1525879831.015811|CUmrqr4svHuSXJy5z7|192.168.100.103|    51524| 65.127.233.163|       23|  tcp|      -|2.999051|         0|         0|        S0|  

In [5]:
## Pre-processing

In [7]:
filepaths = "../data/CTU-IoT-Malware-Capture-1-1conn.log.labeled.csv"
spark = SparkSession.builder.appName("anomaly_detection").getOrCreate()
df = spark.read.option('delimiter', "|").csv(filepaths, header = True)
df = df.withColumn("dt", F.from_unixtime("ts")).withColumn("dt", F.to_timestamp("dt"))  

df = df.withColumnsRenamed(
    {
        "id.orig_h": "source_ip",
        "id.orig_p": "source_port",
        "id.resp_h": "dest_ip",
        "id.resp_p": "dest_port",
    }
)

In [8]:
df.schema

StructType([StructField('ts', StringType(), True), StructField('uid', StringType(), True), StructField('source_ip', StringType(), True), StructField('source_port', StringType(), True), StructField('dest_ip', StringType(), True), StructField('dest_port', StringType(), True), StructField('proto', StringType(), True), StructField('service', StringType(), True), StructField('duration', StringType(), True), StructField('orig_bytes', StringType(), True), StructField('resp_bytes', StringType(), True), StructField('conn_state', StringType(), True), StructField('local_orig', StringType(), True), StructField('local_resp', StringType(), True), StructField('missed_bytes', StringType(), True), StructField('history', StringType(), True), StructField('orig_pkts', StringType(), True), StructField('orig_ip_bytes', StringType(), True), StructField('resp_pkts', StringType(), True), StructField('resp_ip_bytes', StringType(), True), StructField('tunnel_parents', StringType(), True), StructField('label', St

In [9]:
df.show()

+-----------------+------------------+---------------+-----------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+-------------------+
|               ts|               uid|      source_ip|source_port|        dest_ip|dest_port|proto|service|duration|orig_bytes|resp_bytes|conn_state|local_orig|local_resp|missed_bytes|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|tunnel_parents|    label|      detailed-label|                 dt|
+-----------------+------------------+---------------+-----------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+-------------------+
|1525879831.015811|CUmrqr4svHuSXJy5z7|192.168.100.103|      51524| 65.127.233.163|

In [10]:
df.agg(
    F.min("dt").alias("min_date"),
    F.max("dt").alias("max_date")    
).show()

+-------------------+-------------------+
|           min_date|           max_date|
+-------------------+-------------------+
|2018-05-09 16:30:31|2018-05-14 08:24:43|
+-------------------+-------------------+



In [11]:
to_analyse = df.columns
to_analyse = to_analyse[2:-1]

unique_counts = df.agg(*(F.countDistinct(F.col(c)).alias(c) for c in to_analyse))
# * unpacks the element in a list

unique_counts.show()
unique_counts = unique_counts.first()

static_cols = [c for c in unique_counts.asDict() if unique_counts[c] == 1]
print("Static cols in this dataset are {}".format(static_cols))

+---------+-----------+-------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+-----+--------------+
|source_ip|source_port|dest_ip|dest_port|proto|service|duration|orig_bytes|resp_bytes|conn_state|local_orig|local_resp|missed_bytes|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|tunnel_parents|label|detailed-label|
+---------+-----------+-------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+-----+--------------+
|    15004|      32696| 597107|    75844|    3|      5|   16650|       171|       479|        11|         1|         1|           2|    126|       60|         1262|       70|         1142|             1|    2|             3|
+---------+-----------+-------+---------+-----+-------+--------+----------+----------+----------+---

Static cols in this dataset are ['local_orig', 'local_resp', 'tunnel_parents']


In [12]:
filepaths = "../data/CTU-IoT-Malware-Capture-1-1conn.log.labeled.csv"
spark = SparkSession.builder.appName("anomaly_detection").getOrCreate()
df = spark.read.option('delimiter', "|").csv(filepaths, header = True)
df = df.withColumn("dt", F.from_unixtime("ts")).withColumn("dt", F.to_timestamp("dt"))  

df = df.withColumnsRenamed(
    {
        "id.orig_h": "source_ip",
        "id.orig_p": "source_port",
        "id.resp_h": "destination_ip",
        "id.resp_p": "destination_port",
    }
)
df.drop(*static_cols)

DataFrame[ts: string, uid: string, source_ip: string, source_port: string, dest_ip: string, dest_port: string, proto: string, service: string, duration: string, orig_bytes: string, resp_bytes: string, conn_state: string, missed_bytes: string, history: string, orig_pkts: string, orig_ip_bytes: string, resp_pkts: string, resp_ip_bytes: string, label: string, detailed-label: string, dt: timestamp]

In [15]:
df = df.replace("-", None)
remaining_cols = [f for f in to_analyse if f not in static_cols]

df.select(
    [F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in remaining_cols]
).show()

+---------+-----------+-------+---------+-----+-------+--------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+-----+--------------+
|source_ip|source_port|dest_ip|dest_port|proto|service|duration|orig_bytes|resp_bytes|conn_state|missed_bytes|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|label|detailed-label|
+---------+-----------+-------+---------+-----+-------+--------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+-----+--------------+
|        0|          0|      0|        0|    0|1005507|  796300|    796300|    796300|         0|           0|  17421|        0|            0|        0|            0|    0|        469275|
+---------+-----------+-------+---------+-----+-------+--------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+-----+--------------+



In [49]:
numerical_cols = [
    "duration",
    "orig_bytes",
    "resp_bytes",
    "orig_pkts",
    "orig_ip_bytes",
    "resp_pkts",
    "resp_ip_bytes"]

categorical_cols = ["proto", "service", "conn_state"]
label = "label"

all_cols = numerical_cols + categorical_cols

In [62]:
df.schema

StructType([StructField('ts', StringType(), True), StructField('uid', StringType(), True), StructField('source_ip', StringType(), True), StructField('source_port', StringType(), True), StructField('dest_ip', StringType(), True), StructField('dest_port', StringType(), True), StructField('proto', StringType(), True), StructField('service', StringType(), True), StructField('duration', DoubleType(), True), StructField('orig_bytes', DoubleType(), True), StructField('resp_bytes', DoubleType(), True), StructField('conn_state', StringType(), True), StructField('local_orig', StringType(), True), StructField('local_resp', StringType(), True), StructField('missed_bytes', StringType(), True), StructField('history', StringType(), True), StructField('orig_pkts', DoubleType(), True), StructField('orig_ip_bytes', DoubleType(), True), StructField('resp_pkts', DoubleType(), True), StructField('resp_ip_bytes', DoubleType(), True), StructField('tunnel_parents', StringType(), True), StructField('label', St

## Data Pre-processing Pipeline

In [82]:
filepaths = "../data/CTU-IoT-Malware-Capture-1-1conn.log.labeled.csv"
spark = SparkSession.builder.appName("anomaly_detection").getOrCreate()

static_cols = ["local_orig", "local_resp", "missed_bytes", "tunnel_parents"]
numerical_cols = [
    "duration",
    "orig_bytes",
    "resp_bytes",
    "orig_pkts",
    "orig_ip_bytes",
    "resp_pkts",
    "resp_ip_bytes"]

categorical_cols = ["proto", "service", "conn_state"]

preproc_df = spark.read.option('delimiter', "|").csv(filepaths, header = True)
preproc_df = preproc_df.withColumn(
                    "dt", F.from_unixtime("ts"))\
                    .withColumn("dt", F.to_timestamp("dt"))\
        .withColumnsRenamed(
                    {
                        "id.orig_h": "source_ip",
                        "id.orig_p": "source_port",
                        "id.resp_h": "dest_ip",
                        "id.resp_p": "dest_port",
                    })\
        .withColumns(
                {
                        "day": F.date_trunc("day", F.col("dt")),
                        "hour": F.date_trunc("hour", F.col("dt")),
                        "minute": F.date_trunc("minute", F.col("dt")),
                        "second": F.date_trunc("second", F.col("dt"))
                })\
        .drop(*static_cols)\
        .replace("-", None)\
        .withColumns({x: F.col(x).cast("double") for x in numerical_cols})\
        .fillna({x: 'missing' for x in categorical_cols})\
        .fillna({x: -999999 for x in numerical_cols})

preproc_df.show()

+-----------------+------------------+---------------+-----------+---------------+---------+-----+-------+---------+----------+----------+----------+-------+---------+-------------+---------+-------------+---------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|               ts|               uid|      source_ip|source_port|        dest_ip|dest_port|proto|service| duration|orig_bytes|resp_bytes|conn_state|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|    label|      detailed-label|                 dt|                day|               hour|             minute|             second|
+-----------------+------------------+---------------+-----------+---------------+---------+-----+-------+---------+----------+----------+----------+-------+---------+-------------+---------+-------------+---------+--------------------+-------------------+-------------------+-------------------+-------------------+----------